In [2]:
# Standard Imports

import numpy as np
import pandas as pd
import os
import sys
from collections import defaultdict
from importlib import reload
from bs4 import BeautifulSoup
import requests

import matplotlib.pyplot as plt
%matplotlib inline
%config InlineBackend.figure_format = 'retina'


# Load MongoDB

from pymongo import MongoClient
client = MongoClient()
# Access/Initiate Database
db = client['producer_db']
# Access/Initiate Table
tab = db['songs']
collection = db.tab

# Authorize Spotify API

import spotipy
from spotipy.oauth2 import SpotifyClientCredentials

client_id = os.environ['SPOTIFY_CLIENT_ID']
client_secret = os.environ['SPOTIFY_CLIENT_SECRET']
client_credentials_manager = SpotifyClientCredentials(client_id=client_id, client_secret=client_secret)
sp = spotipy.Spotify(client_credentials_manager=client_credentials_manager)

In [9]:
len(collection.find_one()['audio_analysis']['segments'])

498

### Gameplan: 

Make a `timbre` array for every song of size (12,100) by taking the first 100 segments. Make a dictionary where...

In [13]:
test_dict = {'colA':[1,2,3],
             'colB':[4,5,6],
             'colC':['now','we are','done'],
             'colD':[np.array([1,2,3]),np.array([1,2,3]),np.array([1,2,3])]}

pd.DataFrame(test_dict)['colD'][0]

array([1, 2, 3])

In [17]:
track = collection.find_one()['track']
artist = collection.find_one()['artist']
producer = collection.find_one()['producer']

In [19]:
timbre_list = []

for i in range(100):
    timbre_list.append(collection.find_one()['audio_analysis']['segments'][i]['timbre'])
    
timbre = np.stack(timbre_list)

In [22]:
timbre.flatten().shape

(1200,)

Let's see if we can use PCA to distinguish between George Martin (ct. 130) and Rick Rubin (ct. 155)

In [32]:
gm = collection.find({'producer':'George Martin'})
rr = collection.find({'producer':'Rick Rubin'})

In [34]:
tracks = []
artists = []
producers = []
timbres = []
collections = [gm, rr]

for collection in collections:
    for song in collection:
        track = song['track']
        artist = song['artist']
        producer = song['producer']

        timbre_list=[]
        for i in range(80):
            timbre_list.append(song['audio_analysis']['segments'][i]['timbre'])
        timbre = np.concatenate(timbre_list, axis=0)
    
        tracks.append(track)
        artists.append(artist)
        producers.append(producer)
        timbres.append(timbre)
        
producer_dict = {}
producer_dict['track'] = tracks
producer_dict['artist'] = artists
producer_dict['producer'] = producers
producer_dict['timbre'] = timbres

producer_df = pd.DataFrame(producer_dict)

producer_df.head()

,track,artist,producer,timbre
0,Here Comes the Sun,the Beatles,George Martin,"[0.0, 171.13, 9.469, -28.48, 57.491, -50.067, ..."
1,Here Today,Paul McCartney,George Martin,"[2.061, 134.583, 6.964, -64.095, 54.755, -47.1..."
2,"Here, There and Everywhere",the Beatles,George Martin,"[0.0, 171.13, 9.469, -28.48, 57.491, -50.067, ..."
3,Hey Bulldog,the Beatles,George Martin,"[0.0, 171.13, 9.469, -28.48, 57.491, -50.067, ..."
4,Hey Jude,the Beatles,George Martin,"[30.453, 132.462, 82.292, -312.805, 35.647, 22..."


In [37]:
producer_df.producer.value_counts()

Rick Rubin       155
George Martin     61
Name: producer, dtype: int64

In [30]:
a = [1,2,3]
b = [4,5,6]

np.concatenate([a,b,a],axis=0)

array([1, 2, 3, 4, 5, 6, 1, 2, 3])